# Notebook: TCN Model Training and Evaluation

In this notebook, we will load the stock data, preprocess it, build and train a TCN-based model (using our hybrid functions), and then evaluate and plot the results. All functions are imported from the `DL_TCN` module.

## 1. Importing Libraries and Module

First, we import the required libraries and the module containing our functions.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import layers, models, Input, Model, regularizers
from tensorflow.keras.layers import Dropout
from tcn import TCN

In [ ]:
def R2(y_true, y_pred):
    ss_res = tf.reduce_sum(
        tf.square(y_true - y_pred)
    )
    ss_tot = tf.reduce_sum(
        tf.square(y_true - tf.reduce_mean(y_true))
    )
    
    return 1 - ss_res / ss_tot


def ACCURACY_5(y_true, y_pred):
    # Erreur relative : |y_true - y_pred| / (|y_true| + epsilon) on evite les divisions par 0
    error = tf.abs(
        (y_true - y_pred) / (tf.abs(y_true))
    )
    correct = tf.cast(
        error <= 0.05,
        tf.float32
    )

    return tf.reduce_mean(correct)

def CREATE_SEQUENCES(values, sequence_length=60):
    X, y = [], []
    
    for i in range(sequence_length, len(values)):
        X_window = values[i - sequence_length: i]
        y_value = values[i] 
        X.append(X_window)
        y.append(y_value)
        
    X = np.array(X)
    y = np.array(y).reshape(-1, 1)
    
    return X, y

def NN_MODEL(input_shape, learning_rate=0.0005):
    model = models.Sequential([
        layers.Input(shape=input_shape),

        TCN(
            nb_filters=1,
            kernel_size=200,
            nb_stacks=1,
            dilations=[1, 2, 4, 8],
            padding='causal',
            dropout_rate=0.2,
            return_sequences=True
        ),

        layers.LSTM(10),
        Dropout(0.2),

        layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='mean_squared_error',
        metrics=[
            'mean_absolute_error',
            R2,
            ACCURACY_5
        ]
    )

    return model

def TCN_2(input_shape, num_filters=32, kernel_size=2, dropout_rate=0.2):
    inputs = tf.keras.Input(shape=input_shape)
    
    x = layers.Conv1D(
        filters=num_filters,
        kernel_size=kernel_size,
        padding='causal',
        activation='relu'
    )(inputs)
    x = layers.Dropout(dropout_rate)(x)
    
    x = layers.Conv1D(
        filters=num_filters,
        kernel_size=kernel_size,
        padding='causal',
        activation='relu'
    )(x)
    x = layers.Dropout(dropout_rate)(x)
    
    # Connexion résiduelle
    if inputs.shape[-1] != x.shape[-1]:
        inputs_res = layers.Conv1D(num_filters, kernel_size=1, padding='same')(inputs)
    else:
        inputs_res = inputs
    
    x = layers.Add()([inputs_res, x])
    x = layers.Activation('relu')(x)
    
    model = Model(inputs, x, name="TCN_2")
    return model

def NN_MODEL_2(input_shape, tcn_filters=32, lstm_units=50, dropout_rate=0.2, learning_rate=0.0005):
    inputs = Input(shape=input_shape)
    
    tcn_out = TCN_2(input_shape=input_shape,
                              num_filters=tcn_filters,
                              kernel_size=2,
                              dropout_rate=dropout_rate)(inputs)
    
    lstm_out = layers.LSTM(lstm_units)(tcn_out)
    
    outputs = layers.Dense(1, activation='linear')(lstm_out)
    
    model = Model(inputs, outputs, name="Hybrid_TCN_LSTM_Model")
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

## 2. Data Loading

We load our stock market data from a CSV file and extract the "close" price column.


In [ ]:
# Load CSV data
df = pd.read_csv("/Datasets/NASDAQ_100.csv")

# Extract the 'close' column and reshape to (num_samples, 1)
close_prices = df['close'].values.reshape(-1, 1)

print("Data shape:", close_prices.shape)


## 3. Data Preprocessing

Next, we normalize the close prices using a MinMaxScaler.


In [ ]:
# Initialize and fit the scaler
scaler = StandardScaler()
scaled_prices = scaler.fit_transform(close_prices)

# Check a few values
print("Scaled prices (first 5 rows):")
scaled_prices[:5]


## 4. Creating Sequences

We use a sliding window approach to create input sequences and corresponding targets.
Here, we use a sequence length of 1000 days to predict the following day.


In [ ]:
# Set sequence length (number of input days)
sequence_length = 2000

# Create sequences using the function from DL_TCN
X, y = CREATE_SEQUENCES(scaled_prices, sequence_length=sequence_length)
print("X shape:", X.shape)
print("y shape:", y.shape)


## 5. Splitting the Dataset

We split the data into training (70%), validation (15%), and test (15%) sets.


In [ ]:
# Determine split sizes
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.15)

# Create train, validation, and test splits
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

print("Train shapes:", X_train.shape, y_train.shape)
print("Validation shapes:", X_val.shape, y_val.shape)
print("Test shapes:", X_test.shape, y_test.shape)


## 6. Building the Model

We build our hybrid TCN+LSTM model using the function from the module.  
Our model takes sequences of shape (sequence_length, 1) and predicts a single value.


In [ ]:
# Build the model using the function from DL_TCN
model = NN_MODEL(input_shape=(sequence_length, 1), learning_rate=0.0005)
model.summary()


## 7. Training the Model

We train the model using EarlyStopping and ReduceLROnPlateau to avoid overfitting.


In [ ]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[
        #early_stopping,
        reduce_lr
    ]
)


## 8. Plotting the Training Loss

We now plot the training and validation loss curves (MSE) over epochs.


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.title('Loss over Epochs (TCN Model)')
plt.legend()
plt.show()


## 9. Model Evaluation and Predictions

Next, we evaluate the model on the test set and predict closing prices.


In [ ]:
# Evaluate on test set
test_loss = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)

# Make predictions on the test set
y_pred_scaled = model.predict(X_test)


## 10. Inverse Scaling and Metrics Calculation

We inverse the scaling to get the original closing price values, then calculate performance metrics (R², MAE, etc.).


In [ ]:
# Inverse transform the predictions and true values
y_test_unscaled = scaler.inverse_transform(y_test)
y_pred_unscaled = scaler.inverse_transform(y_pred_scaled)

# Calculate metrics using functions from scikit-learn
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error

rmse = np.sqrt(np.mean((y_test_unscaled - y_pred_unscaled) ** 2))
mae = mean_absolute_error(y_test_unscaled, y_pred_unscaled)
r2 = r2_score(y_test_unscaled, y_pred_unscaled)
mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled) * 100
accuracy = ACCURACY_5(y_test_unscaled, y_pred_unscaled)

print("----- Test Set Performance -----")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")
print(f"Accuracy (5% tolerance): {accuracy:.2f}%")
print(f"MAPE: {mape:.2f}%")


## 11. Plotting Predictions vs. Real Values

Finally, we plot the predictions against the real closing prices for the first 100 test samples.


In [ ]:
n_plot = 100
plt.figure(figsize=(10, 5))
plt.plot(y_test_unscaled[:n_plot], label='Real Price')
plt.plot(y_pred_unscaled[:n_plot], label='Predicted Price')
plt.xlabel("Test Sample Index")
plt.ylabel("Closing Price")
plt.title("TCN Model Predictions vs. Real Values")
plt.legend()
plt.show()
